# Code free Autogluon test
 
### 사전 준비 (CloudFormation)

<span style="color:red">NOTE : 본 노트북 코드의 실행에 앞서, 아래 블로그에서 CloudFormation 링크를 이용하여 리소스를 생성합니다. 작업시 버킷이름을 유니크하게 구성하고 해당 이름을 복사하여 클립보드에 저장합니다. </span>
- https://aws.amazon.com/ko/blogs/machine-learning/code-free-machine-learning-automl-with-autogluon-amazon-sagemaker-and-aws-lambda/

<span style="color:red">또는 제공된 `CodeFreeAutoML.yaml`파일을 이용하여 CloudFormation 작업을 직접 실행할 수 있습니다.</span>

해당 CloudFormation은 S3 버킷과 람다, SageMaker Role 등을 생성합니다.  

<img src="https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2020/07/30/automl-001.jpg" width="70%">  

CloudFormation 작업이 완료되면 [lambda 콘솔](https://console.aws.amazon.com/lambda/home?region=us-east-1#/functions)로 이동하여 생성된 람다 함수를 살펴봅니다. 
- boto3 SDK의 `create_training_job()`을 사용하여 MXNet 컨테이너를 기반으로 `autogluon-tab-with-test.py` 스크립트를 실행하는 SageMaker 작업을 생성하도록 되어있습니다. 
- `autogluon-tab-with-test.py`파일은 S3 버킷의 `source`폴더에서 가져오도록 구성되었습니다.
- [autogluon-tab-with-test.py](autogluon-tab-with-test.py) 스크립트를 열어 코드 내용을 살펴봅니다.

<span style="color:red">NOTE : CloudFormation을 생성할 때 사용한 버킷명을 변수에 저장합니다.</span>

In [ ]:
BUCKET = 'code-free-automl-us-east-1-yournamehere' # please update the bucket name as yours
assert (BUCKET != 'code-free-automl-us-east-1-yournamehere' ), "Please update the bucket name as yours"

`autogluon-tab-with-test.py` 파일을 `sourcedir.tar.gz`로 압축한 후 해당 버킷의 `source`디렉토리로 복사합니다.

In [ ]:
!tar -zcvf sourcedir.tar.gz autogluon-tab-with-test.py
!aws s3 cp sourcedir.tar.gz s3://{BUCKET}/source/
!aws s3 ls s3://{BUCKET}/source/

### 데이터 준비

Autogluon에서 제공되는 데이터를 이용하기 위해 autogloun을 설치합니다.  
또는 Pandas를 이용할 수도 있습니다.

In [ ]:
# python3 -m pip install "mxnet<2.0.0"
# python3 -m pip install "gluoncv>=0.5.0,<0.9.0"
# python3 -m pip install autogluon

In [ ]:
import pandas as pd
import autogluon as ag


데이터셋을 다운로드합니다.

In [ ]:
train_data = pd.read_csv('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
test_data = pd.read_csv('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')

In [ ]:
print(train_data.shape)
print(test_data.shape)

목표변수의 컬럼명을 `target` 으로 수정하고 저장합니다.

In [ ]:
columns = list(train_data.columns[:-1])
columns.append('target')
train_data.columns = columns
test_data.columns = columns

income 이 50K이상인지 이하인지 구별하는 데이터입니다.

In [ ]:
train_data.head(5)

In [ ]:
test_data.head(5)

In [ ]:
train_data.to_csv('income_train.csv')
test_data.to_csv('income_test.csv')


### Training & test 실행 (file upload)
CloudFormation에서 생성한 S3 경로에 파일을 복사합니다. 

In [ ]:
!aws s3 ls s3://{BUCKET}/data/

In [ ]:
!aws s3 cp income_train.csv s3://{BUCKET}/data/income_train.csv
!aws s3 cp income_test.csv s3://{BUCKET}/data/income_test.csv

### 작업실행 모니터링
Lambda와 SageMaker training 작업이 실행되었을 것입니다. 작업은 10분정도 걸립니다. SageMaker Training job이 completed 상태가 되는 것을 기다리면서 다음을 확인합니다. (콘솔 UI를 통해 확인해 봅니다.)
- Lambda 콘솔에서 Lambda 로그를 확인합니다.  
- SageMaker 콘솔에서 Training Job 로그를 확인합니다.

SageMaker SDK에서 제공하는 함수를 통해서도 작업을 모니터링할 수 있습니다. 다음 코드는 현재 실행상태인 작업을 표시합니다. 

In [ ]:
import boto3
sm = boto3.client('sagemaker')

sm.list_training_jobs(StatusEquals='InProgress', SortBy='CreationTime')['TrainingJobSummaries']

가장 최근 제출된 작업명을 기억합니다. (해당 작업명으로 S3에 폴더가 생성됩니다.)

In [ ]:
last_sm_job_name = sm.list_training_jobs(SortBy='CreationTime')['TrainingJobSummaries'][0]['TrainingJobName']
last_sm_job_name

아래 코드는 비동기로 제출된 작업을 노트북에서 동기모드로 모니터링하게 해 줍니다. 현재 실행중인 작업로그를 노트북에서 확인할 수 있습니다. 

In [ ]:
import sagemaker

sagemaker.Session().logs_for_job(job_name=last_sm_job_name, wait=True)


### 추론결과 확인
SageMaker Training job이 completed 상태가 된 후 S3 results 경로를 확인하면 Training job 이름을 가지는 폴더가 생성되었을 것입니다.

In [ ]:
!aws s3 ls s3://{BUCKET}/results/{last_sm_job_name}/

해당 폴더의 파일을 복사하여 가져옵니다.

In [ ]:
!aws s3 cp s3://{BUCKET}/results/{last_sm_job_name}/income_leaderboard.csv income_leaderboard.csv
!aws s3 cp s3://{BUCKET}/results/{last_sm_job_name}/income_test_predictions.csv income_test_predictions.csv
    

In [ ]:
leaderboard = pd.read_csv('income_leaderboard.csv')
test_predictions = pd.read_csv('income_test_predictions.csv')

leaderboard 파일에 automl실행 결과가 저장되어 있습니다.

In [ ]:
leaderboard

test_predictions 파일에 테스트데이터에 대한 예측결과가 저장되어 있습니다.

In [ ]:
test_predictions.head()